In [1]:
import copy
import datetime
import h5py
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import sys

from cv2 import resize
from datetime import datetime
from gc import collect
from os import cpu_count
from sklearn.model_selection import train_test_split
from time import sleep
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
from tqdm import tqdm

sys.path.append(f"{os.getcwd()}/ViT-TFM/")
sys.path.append(f"{os.getcwd()}/mltfm/")
import VisionTransformer as vit

2022-11-30 09:54:18.092722: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 09:54:18.183772: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-30 09:54:18.183789: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-30 09:54:18.207687: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-30 09:54:18.637248: W tensorflow/stream_executor/platform/de

In [2]:
random_seed = 1
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
cudnn.benchmark = True

In [3]:
collect()
torch.cuda.empty_cache()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Running on device: {device}")

Running on device: cuda


In [4]:
vit_weights = vit.VisionTransformer(dspl_size=104, patch_size=8, embed_dim=128, depth=12, n_heads=8, mlp_ratio=4.,p=0., attn_p=0.,drop_path=0).double()

In [5]:
path_to_pth = 'ViT-TFM/logs_and_weights/ViT-2022-Nov-24 20:44:29/ViT-2022-Nov-24 20:44:29_best_val_loss_0.000204.pth'

In [6]:
if torch.cuda.is_available():
    vit_weights.load_state_dict(torch.load(path_to_pth)['best_model_weights'], strict=False)
else:
    vit_weights.load_state_dict(torch.load(path_to_pth, map_location=torch.device('cpu'))['best_model_weights'], strict=False)        

FileNotFoundError: [Errno 2] No such file or directory: 'ViT-TFM/logs_and_weights/ViT-2022-Nov-24 20:44:29/ViT-2022-Nov-24 20:44:29_best_val_loss_0.000204.pth'

## 1. [Gaussian process layers](https://arxiv.org/pdf/2106.00638.pdf)

__Goal__: Use variance of GP as distance measure between new input and already observed training data.

__Idea__: 
  1. Combination of Deep Neural Networks (DNNs) and Gaussian Processes (GPs)
  2. Scalable Variational Gaussian Processes (SVGPs): Complexity of O(m³) with m as number of inducing points
  3. Use the outputs of a neural network as the input to an uncertainty aware GP
  4. Given a new input sample, the GP model provides a predictive distribution over (univariate!) target variable

__Algorithm__: 
  1. Train backbone (e.g. CNN, Transformer) on some problem
  2. Generate inducing points by feeding images into the model and embed them
  3. Use these embedding points as inducing points for SVGP
  4. Maximize SVGP parameters (e.g. scaling parameter of RBF kernel) by maximizing ELBO

__Problem__: For every target variable, one SVGP has to be trained. Furthermore, SVGPs assume independence between the target variables.

## 2. Distance in latent space

"What the neural net is essentially doing is stripping away details from the features that are not helpful and amplifying the most important ones, and by the time we get to the latent space, it’s basically the way the model sees the molecule to make the prediction."

In [7]:
dspl = h5py.File('mltfm/displacements_res_104_num_50000.h5')["data"]

In [8]:
dspl_ae = np.array(dspl[0:110, :, :, :])
dspl_vit = np.moveaxis(np.array(dspl_ae),3 ,1)

X_train_ae = torch.from_numpy(dspl_ae).double()
X_test_vit = torch.from_numpy(dspl_vit).double()

In [9]:
class LatentRepresentationExtractor(nn.Module):
    def __init__(self, patch_embed, pos_embed, blocks):
        super().__init__()
        self.patch_embed = patch_embed
        self.pos_embed = pos_embed
        self.blocks = blocks
        
    def forward(self, x):
        x = self.patch_embed(x)
        x = x + self.pos_embed  # (n_samples, n_patches, embed_dim)

        for block in self.blocks:
            x, _ = block(x)
        
        return x

In [10]:
representation_extractor = LatentRepresentationExtractor(vit_weights.patch_embed, vit_weights.pos_embed, vit_weights.blocks)

In [46]:
representations = representation_extractor(X_test_vit)

In [50]:
representations = representations.detach()

In [51]:
class DeepAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(10 * 169 , 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 8)
        )
        
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(8, 32),
        torch.nn.ReLU(),
        torch.nn.Linear(32, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128, 10 * 169),
        )

    def forward(self, x):
        x = x.reshape(-1, 10 * 169)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [52]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = DeepAutoencoder().to(device).double()

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = nn.MSELoss()

In [53]:
X_train = representations[0:100, :, :].double()
X_val = representations[100:, :, :].double()

train_set = TensorDataset(X_train, X_train)
val_set = TensorDataset(X_val, X_val)

dataloader_kwargs = {'batch_size': 10,
                     'num_workers': 10,
                     'pin_memory': False,
                     'shuffle': True}
dataloaders = {'train': DataLoader(train_set, **dataloader_kwargs),
               'val': DataLoader(val_set, **dataloader_kwargs)}

In [54]:
def run_epoch(model, loss_fn, dataloader, device, epoch, optimizer, train, visualize_attn=False):
    # Set model to training mode
    if train:
        model.train()
    else:
        model.eval()

    epoch_loss = 0.0

    with tqdm(dataloader, unit="batch") as tepoch:
        # Iterate over data
        for xb, yb in tepoch:
            tepoch.set_description(f"Epoch {epoch}")

            xb, yb = xb.to(device), yb.to(device)
            yb = yb.reshape(-1, 10 * 169)

            # zero the parameters
            if train:
                optimizer.zero_grad(set_to_none=True)

            # forward
            if visualize_attn:
                pred, attn_scores = model(xb)  # attn_scores: [attn_1, ..., attn_depth]
            else:
                pred = model(xb)
            if not train and visualize_attn:
                with torch.set_grad_enabled(not train):
                    attn_mat = torch.stack(attn_scores) # attn_mat.shape==(depth, samples, n_heads, n_patches**0.5, n_patches**0.5)
                    attn_mat = attn_mat.squeeze(1) # shape remains unchanged unless samples==1, then: (depth, n_heads, n_patches**0.5, n_patches**0.5)
                    attn_mat = torch.mean(attn_mat, dim=1) # if samples==1: attn_mat.shape==(depth, n_patches**0.5,n_patches**0.5)

            with torch.set_grad_enabled(train):
                loss = loss_fn(pred, yb)

                # backward + optimize if in training phase
                if train:
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type=2)
                    optimizer.step()

            # statistics
            epoch_loss += loss.item()

        epoch_loss /= len(dataloader.dataset)
        tepoch.set_postfix(loss=epoch_loss)
        sleep(0.01)
    return epoch_loss

In [55]:
def fit(model, loss_fn, dataloaders, optimizer, device, writer, NAME, max_epochs, patience):
    best_val_loss = np.inf
    best_epoch = -1
    best_model_weights = {}
    optimizer_state_dict_in_best_epoch = {}
    model.to(device)
    example_input, example_value = next(iter(dataloaders['train']))
    writer.add_graph(model, example_input.to(device))

    for epoch in range(1, max_epochs + 1):
        train_loss = run_epoch(model, loss_fn, dataloaders['train'], device, epoch, optimizer, train=True)
        val_loss = run_epoch(model, loss_fn, dataloaders['val'], device, epoch, optimizer=None, train=False)
        print(
            f"Epoch {epoch}/{max_epochs}, train_loss: {train_loss}")

        writer.add_scalar('train_loss', train_loss, epoch)
        writer.add_scalar('val_loss', val_loss, epoch)

        # Save best weights
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            best_model_weights = copy.deepcopy(model.state_dict())
            optimizer_state_dict_in_best_epoch = copy.deepcopy(optimizer.state_dict())

        # Early stopping
        print(
            f"best_val_loss: {np.round(best_val_loss, 6)}, "
            f"epoch: {epoch}, best_epoch: {best_epoch}, "
            f"current_patience: {patience - (epoch - best_epoch)}")
        if epoch - best_epoch >= patience:
            break

    torch.save({
        'best_epoch': best_epoch,
        'best_model_weights': best_model_weights,
        'optimizer_state_dict_in_best_epoch': optimizer_state_dict_in_best_epoch,
        'best_val_loss': best_val_loss
    },
        f'logs_and_weights/{NAME}/{NAME}_best_val_loss_{np.round(best_val_loss, 6)}.pth')



In [56]:
NAME = "AE-{:%Y-%b-%d %H:%M:%S}".format(datetime.now())
writer = SummaryWriter(log_dir=f'logs_and_weights/{NAME}')
fit(model, criterion, dataloaders, optimizer, device, writer, NAME, 100, 20)
writer.close()

Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.65s/batch]


Epoch 1/100, train_loss: 13165307.046131387
best_val_loss: 12974643.249953, epoch: 1, best_epoch: 1, current_patience: 20


Epoch 2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.62s/batch]


Epoch 2/100, train_loss: 12800090.42457653
best_val_loss: 12384418.159308, epoch: 2, best_epoch: 2, current_patience: 20


Epoch 3: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.66s/batch]


...

Epoch 98/100, train_loss: 193007.98674856802
best_val_loss: 194679.23291, epoch: 98, best_epoch: 84, current_patience: 6


Epoch 99: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.62s/batch]


Epoch 99/100, train_loss: 193409.11364959655
best_val_loss: 194679.23291, epoch: 99, best_epoch: 84, current_patience: 5


Epoch 100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.68s/batch]

Epoch 100/100, train_loss: 197945.78299172653
best_val_loss: 194679.23291, epoch: 100, best_epoch: 84, current_patience: 4


Compute whether embedding of new sample is within convex hull of training samples.

In [58]:
def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    from scipy.spatial import Delaunay
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

In [59]:
new_point = dspl[111, :, :, :]

In [61]:
new_point = np.moveaxis(np.array(new_point[np.newaxis]),3 ,1)

In [62]:
new_point = torch.from_numpy(new_point).double()

In [63]:
new_point_rep = representation_extractor(new_point)